In [2]:
!pip install --upgrade pip
!pip install --upgrade datasets transformers accelerate soundfile librosa evaluate jiwer tensorboard gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 4.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.1
    Uninstalling pip-23.3.1:
      Successfully uninstalled pip-23.3.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 4.4 MB/s eta 0:00:00
  Using cached tensorboard-2.16.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.5 MB/s eta 0:00:00
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached Markdown-3.5.2-py3-none-any.whl.metadata (7.0 kB)
  Using cached protobuf-4.25.3-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
  Using cached tensorboard_data_server-0.7.2-py3-none-macosx_10_9_x86_64.whl.metadata (1.1 kB)
  Using cached w

In [73]:
from huggingface_hub import interpreter_login
from datasets import load_dataset, DatasetDict, Audio
from transformers import WhisperFeatureExtractor, WhisperProcessor, WhisperTokenizer, WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
import time
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate
import torch

In [4]:
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token is valid (permission: write).
Your token has been saved to /Users/amyguan/.cache/huggingface/token
Login successful


In [85]:
dataset = load_dataset("mozilla-foundation/common_voice_16_1", "en", split="train", token=True, trust_remote_code=True, streaming=True)
dataset.features

{'client_id': Value(dtype='string', id=None),
 'path': Value(dtype='string', id=None),
 'audio': Audio(sampling_rate=48000, mono=True, decode=True, id=None),
 'sentence': Value(dtype='string', id=None),
 'up_votes': Value(dtype='int64', id=None),
 'down_votes': Value(dtype='int64', id=None),
 'age': Value(dtype='string', id=None),
 'gender': Value(dtype='string', id=None),
 'accent': Value(dtype='string', id=None),
 'locale': Value(dtype='string', id=None),
 'segment': Value(dtype='string', id=None),
 'variant': Value(dtype='string', id=None)}

In [86]:
dataset = dataset.shuffle(seed=42, buffer_size=10_000)
dataset = dataset.take(60_000) # approx half of training
dataset = dataset.filter(lambda example: example['accent'] != '') # doesn't work
dataset = dataset.cast_column("audio", Audio(sampling_rate=16_000))

# Preprocess and Init Whisper

In [84]:
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="English", task="transcribe")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [88]:
metric = evaluate.load("wer")
dialects = {}

for example in dataset:
    accents = example["accent"].split(',')  # might mess up the "India and South Asia (India, Pakistan, Sri Lanka)", but fix later
    if not accents: # idk why the filter for empties over the whole dataset isn't working :(
        continue
    for accent in accents:
        if accent not in dialects:
            dialects[accent] = []

    audio_sample = example["audio"]["array"]
    sampling_rate = example["audio"]["sampling_rate"]
    input_features = processor(
        audio_sample, sampling_rate=sampling_rate, return_tensors="pt"
    ).input_features

    start = time.time()
    predicted_ids = model.generate(input_features)
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
    # print(f'transcription: {transcription[0]}')
    # print(f'reference sentence: {example["sentence"]}')
    wer = metric.compute(predictions=[transcription[0]], references=[example["sentence"]])
    # print(wer)
    for accent in accents:
        dialects[accent].append(wer)

# convert to tensor?

Reading metadata...: 641344it [00:19, 33243.53it/s]


KeyboardInterrupt: 

In [83]:
print(dialects.keys())

dict_keys(['', 'Scottish English', 'Canadian English', 'India and South Asia (India, Pakistan, Sri Lanka)', 'United States English', 'Singaporean English', 'England English', 'United States English,Midwestern States (Michigan)', 'Welsh English', 'Filipino', 'Australian English', 'New Zealand English', 'Irish English', 'United States English,Central American ', 'Malaysian English', 'England English,British', 'Slavic', 'German', 'Kazakhstan English', 'India and South Asia (India, Pakistan, Sri Lanka),International Indian Accent', 'Slavic,Russian', 'Mid-atlantic,England English,United States English', 'United States English,England English'])


In [ ]:
# # split per dialect?

# def prepare_dataset(batch):
#     audio = batch["audio"]
#     # log-Mel
#     batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
#     # compute input length of audio sample in seconds
#     batch["input_length"] = len(audio["array"]) / audio["sampling_rate"]

#     # process targets
#     input_str = batch["sentence"]
#     # encode target text to label ids
#     batch["labels"] = processor.tokenizer(input_str).input_ids

#     return batch

In [ ]:
# vectorized_datasets = dataset.map(
#     prepare_dataset,
#     num_proc=preprocessing_num_workers,
#     remove_columns=next(iter(dataset.values())).column_names,
#     desc="preprocess dataset",
# )

In [ ]:
# max_input_length = 30
# min_input_length = 0

# def is_audio_in_length_range(length):
#     return length > min_input_length and length < max_input_length

# vectorized_datasets = vectorized_datasets.filter(
#     is_audio_in_length_range, num_proc=preprocessing_num_workers, input_columns=["input_length"]
# )

In [ ]:
# max_label_length = model.config.max_length

# def is_labels_in_length_range(labels):
#     return len(labels) < max_label_length

# vectorized_datasets = vectorized_datasets.filter(
#     is_labels_in_length_range, num_proc=preprocessing_num_workers, input_columns=["labels"]
# )

In [ ]:
# @dataclass
# class DataCollatorSpeechSeq2SeqWithPadding:
#     processor: Any

#     def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
#         input_features = [{"input_features": feature["input_features"]} for feature in features]
#         # convert to tensors
#         batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

#         label_features = [{"input_ids": feature["labels"]} for feature in features]
#         # pad label ids to the max length in the batch
#         labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

#         # replace padding with -100 to ignore loss correctly
#         labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

#         # if bos token is appended in previous tokenization step,
#         # cut bos token here as it's append later anyways
#         if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
#             labels = labels[:, 1:]

#         batch["labels"] = labels

#         return batch
    

# data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
# metric = evaluate.load("wer")

# evaluate with the 'normalized' WER
# do_normalize_eval = True


# def compute_metrics(pred):
#     pred_ids = pred.predictions
#     label_ids = pred.label_ids

#     # replace -100 with the pad_token_id
#     label_ids[label_ids == -100] = tokenizer.pad_token_id

#     # we do not want to group tokens when computing the metrics
#     pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
#     label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

#     if do_normalize_eval:
#         # filtering step to only evaluate the samples that correspond to non-zero references
#         pred_str = [pred_str[i] for i in range(len(pred_str)) if len(label_str[i]) > 0]
#         label_str = [label_str[i] for i in range(len(label_str)) if len(label_str[i]) > 0]

#     wer = metric.compute(predictions=pred_str, references=label_str)

#     return {"wer": wer}

In [ ]:
# # training?/? i feel like i shouldn't need this

# training_args = Seq2SeqTrainingArguments(
#     output_dir="./outputs/whisper_tiny_ft",
#     per_device_train_batch_size=64,
#     per_device_eval_batch_size=32,
#     gradient_accumulation_steps=1,
#     warmup_steps=800,
#     max_steps=8000,
#     learning_rate=6.25e-6,
#     weight_decay=0.01,
#     gradient_checkpointing=True,
#     fp16=True,
#     predict_with_generate=True,
#     generation_max_length=225,
#     logging_steps=25,
#     report_to=["tensorboard"],
#     evaluation_strategy="steps",
#     eval_steps=1000,
#     save_strategy="steps",
#     save_steps=1000,
#     load_best_model_at_end=True,
#     metric_for_best_model="wer",
#     greater_is_better=False,
# )

# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=vectorized_datasets["train"],
#     eval_dataset=vectorized_datasets["eval"],
#     tokenizer=processor,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
# )

# trainer.train()
# trainer.evaluate()
# model.save_pretrained(training_args.output_dir)
# processor.save_pretrained(training_args.output_dir)

In [71]:
# for example in dataset:
#     audio_sample = example["audio"]["array"]
#     sampling_rate = example["audio"]["sampling_rate"]
#     input_features = processor(
#         audio_sample, sampling_rate=sampling_rate, return_tensors="pt"
#     ).input_features

#     start = time.time()
#     predicted_ids = model.generate(input_features)
#     print(f'time: {time.time() - start}')
#     transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
#     print(transcription[0])